In [1]:
# Check Python version (optional):
import sys
print("Python version:", sys.version)

# Get installations
!pip install --quiet torch numpy matplotlib scikit-learn pandas
!pip install --quiet huggingface_hub transformers

import os
import torch
import numpy as np
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA

# If you want to check GPU usage:
import torch
device = "cuda" if torch.cuda.is_available() else "cpu"
print("Using device:", device)

Python version: 3.11.11 (main, Dec  4 2024, 08:55:07) [GCC 11.4.0]
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 60.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 85.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 57.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 9.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 35.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 9.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 93.2 MB/s eta 0:00:00
Using device: cuda


In [2]:
import torch
torch.cuda.empty_cache()
import gc
gc.collect()
%ls

sample_data/


In [3]:
from google.colab import drive
drive.mount('/content/drive')

# After running this cell, follow the link to grant Colab access to your Google Drive.

Mounted at /content/drive


In [4]:
!git clone https://github.com/IdaCy/polAItness_internals.git
%cd polAItness_internals

Cloning into 'polAItness_internals'...
remote: Enumerating objects: 272, done.
remote: Counting objects: 100% (272/272), done.
remote: Compressing objects: 100% (195/195), done.
remote: Total 272 (delta 130), reused 204 (delta 66), pack-reused 0 (from 0)
Receiving objects: 100% (272/272), 417.13 KiB | 4.09 MiB/s, done.
Resolving deltas: 100% (130/130), done.
/content/polAItness_internals


In [5]:
!pip install huggingface_hub --quiet

from huggingface_hub import notebook_login

# This will prompt you in Colab to enter your HF token or log in directly
notebook_login()

In [35]:
!git pull

remote: Enumerating objects: 3, done.
remote: Counting objects: 100% (2/2), done.
remote: Total 3 (delta 2), reused 2 (delta 2), pack-reused 1 (from 1)
Unpacking objects: 100% (3/3), 318 bytes | 318.00 KiB/s, done.
From https://github.com/IdaCy/polAItness_internals
   34bb661..a026640  main       -> origin/main
Updating 34bb661..a026640
Fast-forward
 prompts/levels.json | 11002 ++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
 1 file changed, 11002 insertions(+)
 create mode 100644 prompts/levels.json


In [10]:
import logging
from collab_modules.logging import init_logger
logging.basicConfig(level=logging.INFO)

logger = init_logger(
    log_file="logs/progress.log",
    console_level=logging.WARNING,  # only warnings to console
    file_level=logging.DEBUG        # full debug info in the file
)

In [17]:
from collab_modules.indexed_inference import (
    init_logger,
    load_model,
    load_json_prompts,
    run_inf
)

In [14]:
model, tokenizer = load_model(
    model_name="google/gemma-2-9b-it",
    use_bfloat16=True,
    hf_token=None,
    logger=logger
)

/usr/local/lib/python3.11/dist-packages/transformers/models/auto/tokenization_auto.py:862: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/47.0k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.24M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.5M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/636 [00:00<?, ?B/s]

...

model-00003-of-00004.safetensors:   0%|          | 0.00/4.96G [00:00<?, ?B/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/3.67G [00:00<?, ?B/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/4.90G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/173 [00:00<?, ?B/s]

In [65]:
# Inference on attack_jailbreak

data = load_json_prompts(
    file_path="prompts/levels.json",
    prompt_key="normal",
    max_samples=None,
    logger=logger
)

# 3. Rerun inference
run_inf(
    model=model,
    tokenizer=tokenizer,
    data=data,
    batch_size=128,
    output_dir="/content/drive/MyDrive/polAItness_internals/output/extractions/gemma-2-9b-it/normal",
    extract_hidden_layers=[0, 5, 10, 15, 20, 25],
    extract_attention_layers=[0, 5, 10, 15, 20, 25],
    max_seq_length=2048,
    logger=logger,
    generation_kwargs={
        "do_sample": True,
        "max_new_tokens": 100,
        "temperature": 0.7,
        "top_p": 0.9,
        "repetition_penalty": 1.2
    }
)

In [66]:
# Inference on attack_jailbreak

data = load_json_prompts(
    file_path="prompts/levels.json",
    prompt_key="nicer",
    max_samples=None,
    logger=logger
)

# 3. Rerun inference
run_inf(
    model=model,
    tokenizer=tokenizer,
    data=data,
    batch_size=128,
    output_dir="/content/drive/MyDrive/polAItness_internals/output/extractions/gemma-2-9b-it/nicer",
    extract_hidden_layers=[0, 5, 10, 15, 20, 25],
    extract_attention_layers=[0, 5, 10, 15, 20, 25],
    max_seq_length=2048,
    logger=logger,
    generation_kwargs={
        "do_sample": True,
        "max_new_tokens": 100,
        "temperature": 0.7,
        "top_p": 0.9,
        "repetition_penalty": 1.2
    }
)

In [67]:
# Inference on attack_jailbreak

data = load_json_prompts(
    file_path="prompts/levels.json",
    prompt_key="urgent",
    max_samples=None,
    logger=logger
)

# 3. Rerun inference
run_inf(
    model=model,
    tokenizer=tokenizer,
    data=data,
    batch_size=128,
    output_dir="/content/drive/MyDrive/polAItness_internals/output/extractions/gemma-2-9b-it/urgent",
    extract_hidden_layers=[0, 5, 10, 15, 20, 25],
    extract_attention_layers=[0, 5, 10, 15, 20, 25],
    max_seq_length=2048,
    logger=logger,
    generation_kwargs={
        "do_sample": True,
        "max_new_tokens": 100,
        "temperature": 0.7,
        "top_p": 0.9,
        "repetition_penalty": 1.2
    }
)

In [68]:
# Inference on attack_jailbreak

data = load_json_prompts(
    file_path="prompts/levels.json",
    prompt_key="insulting",
    max_samples=None,
    logger=logger
)

# 3. Rerun inference
run_inf(
    model=model,
    tokenizer=tokenizer,
    data=data,
    batch_size=128,
    output_dir="/content/drive/MyDrive/polAItness_internals/output/extractions/gemma-2-9b-it/insulting",
    extract_hidden_layers=[0, 5, 10, 15, 20, 25],
    extract_attention_layers=[0, 5, 10, 15, 20, 25],
    max_seq_length=2048,
    logger=logger,
    generation_kwargs={
        "do_sample": True,
        "max_new_tokens": 100,
        "temperature": 0.7,
        "top_p": 0.9,
        "repetition_penalty": 1.2
    }
)

In [69]:
# Inference on attack_jailbreak

data = load_json_prompts(
    file_path="prompts/levels.json",
    prompt_key="reduced",
    max_samples=None,
    logger=logger
)

# 3. Rerun inference
run_inf(
    model=model,
    tokenizer=tokenizer,
    data=data,
    batch_size=128,
    output_dir="/content/drive/MyDrive/polAItness_internals/output/extractions/gemma-2-9b-it/reduced",
    extract_hidden_layers=[0, 5, 10, 15, 20, 25],
    extract_attention_layers=[0, 5, 10, 15, 20, 25],
    max_seq_length=2048,
    logger=logger,
    generation_kwargs={
        "do_sample": True,
        "max_new_tokens": 100,
        "temperature": 0.7,
        "top_p": 0.9,
        "repetition_penalty": 1.2
    }
)

In [70]:
# Inference on attack_jailbreak

data = load_json_prompts(
    file_path="prompts/levels.json",
    prompt_key="shortest",
    max_samples=None,
    logger=logger
)

# 3. Rerun inference
run_inf(
    model=model,
    tokenizer=tokenizer,
    data=data,
    batch_size=128,
    output_dir="/content/drive/MyDrive/polAItness_internals/output/extractions/gemma-2-9b-it/shortest",
    extract_hidden_layers=[0, 5, 10, 15, 20, 25],
    extract_attention_layers=[0, 5, 10, 15, 20, 25],
    max_seq_length=2048,
    logger=logger,
    generation_kwargs={
        "do_sample": True,
        "max_new_tokens": 100,
        "temperature": 0.7,
        "top_p": 0.9,
        "repetition_penalty": 1.2
    }
)

In [71]:
# Inference on attack_jailbreak

data = load_json_prompts(
    file_path="prompts/levels.json",
    prompt_key="explained",
    max_samples=None,
    logger=logger
)

# 3. Rerun inference
run_inf(
    model=model,
    tokenizer=tokenizer,
    data=data,
    batch_size=128,
    output_dir="/content/drive/MyDrive/polAItness_internals/output/extractions/gemma-2-9b-it/explained",
    extract_hidden_layers=[0, 5, 10, 15, 20, 25],
    extract_attention_layers=[0, 5, 10, 15, 20, 25],
    max_seq_length=2048,
    logger=logger,
    generation_kwargs={
        "do_sample": True,
        "max_new_tokens": 100,
        "temperature": 0.7,
        "top_p": 0.9,
        "repetition_penalty": 1.2
    }
)

In [72]:
# Inference on attack_jailbreak

data = load_json_prompts(
    file_path="prompts/levels.json",
    prompt_key="pure",
    max_samples=None,
    logger=logger
)

# 3. Rerun inference
run_inf(
    model=model,
    tokenizer=tokenizer,
    data=data,
    batch_size=128,
    output_dir="/content/drive/MyDrive/polAItness_internals/output/extractions/gemma-2-9b-it/pure",
    extract_hidden_layers=[0, 5, 10, 15, 20, 25],
    extract_attention_layers=[0, 5, 10, 15, 20, 25],
    max_seq_length=2048,
    logger=logger,
    generation_kwargs={
        "do_sample": True,
        "max_new_tokens": 100,
        "temperature": 0.7,
        "top_p": 0.9,
        "repetition_penalty": 1.2
    }
)

In [73]:
!ls /content/drive/MyDrive/polAItness_internals/output/extractions/gemma-2-9b-it/

explained  insulting  nicer  normal  pure  reduced  shortest  urgent


In [76]:
!ls /content/drive/MyDrive/polAItness_internals/output/extractions/gemma-2-9b-it/*

/content/drive/MyDrive/polAItness_internals/output/extractions/gemma-2-9b-it/explained:
activations_00000_00128.pt  activations_00384_00512.pt	activations_00768_00896.pt
activations_00128_00256.pt  activations_00512_00640.pt	activations_00896_01000.pt
activations_00256_00384.pt  activations_00640_00768.pt

/content/drive/MyDrive/polAItness_internals/output/extractions/gemma-2-9b-it/insulting:
activations_00000_00128.pt  activations_00384_00512.pt	activations_00768_00896.pt
activations_00128_00256.pt  activations_00512_00640.pt	activations_00896_01000.pt
activations_00256_00384.pt  activations_00640_00768.pt

/content/drive/MyDrive/polAItness_internals/output/extractions/gemma-2-9b-it/nicer:
activations_00000_00128.pt  activations_00384_00512.pt	activations_00768_00896.pt
activations_00128_00256.pt  activations_00512_00640.pt	activations_00896_01000.pt
activations_00256_00384.pt  activations_00640_00768.pt

/content/drive/MyDrive/polAItness_internals/output/extractions/gemma-2-9b-it/nor

In [75]:
!ls /content/drive/MyDrive/polAItness_internals/output/extractions/gemma-2-9b-it/normal | wc -l
!ls /content/drive/MyDrive/polAItness_internals/output/extractions/gemma-2-9b-it/nicer | wc -l
!ls /content/drive/MyDrive/polAItness_internals/output/extractions/gemma-2-9b-it/urgent | wc -l
!ls /content/drive/MyDrive/polAItness_internals/output/extractions/gemma-2-9b-it/insulting | wc -l
!ls /content/drive/MyDrive/polAItness_internals/output/extractions/gemma-2-9b-it/reduced | wc -l
!ls /content/drive/MyDrive/polAItness_internals/output/extractions/gemma-2-9b-it/shortest | wc -l
!ls /content/drive/MyDrive/polAItness_internals/output/extractions/gemma-2-9b-it/explained | wc -l
!ls /content/drive/MyDrive/polAItness_internals/output/extractions/gemma-2-9b-it/pure | wc -l

8
8
8
8
8
8
8
8


## Checking the predictions

In [77]:
import sys
sys.path.append(".")

from collab_modules.read_predictions import read_predictions

# Then call it:
preds = read_predictions(
    read_output_dir="/content/drive/MyDrive/polAItness_internals/output/extractions/gemma-2-9b-it/normal",
    max_predictions=50,
    write_predictions_file="logs/pred_out.txt",
    log_file="logs/readpred.log"
)

print("Number of preds:", len(preds))
print("First 3 preds:", preds[:15])

[INFO] === Starting read_predictions function ===
[INFO] read_output_dir = /content/drive/MyDrive/polAItness_internals/output/extractions/gemma-2-9b-it/normal
[INFO] log_file = logs/readpred.log
[INFO] max_predictions = 50
[INFO] write_predictions_file = logs/pred_out.txt
[INFO] Found 8 .pt files to process.
Reading .pt files:   0%|          | 0/8 [00:00<?, ?it/s]
[INFO] Collected 50 total predictions.
[INFO] === Sample of collected predictions ===
[INFO] Prediction 1: Solve this task: 4 + 93 = ?

**Answer:** 97
[INFO] Prediction 2: Please find the result: 6 - 4 = ?

**Answer:** 2

[INFO] Prediction 3: What is the solution for: 9 - 7 = ?

Here's how to solve it:

* **Start with the numbers:**  You have 9 and 7.
* **Subtract:** Take away 7 from 9.
* **The answer:** 9 - 7 = 2


Let me know if you want to try another problem!
[INFO] Prediction 4: Could you compute: 5 + 2 = ?

**Answer:** 7

[INFO] Prediction 5: I'd like you to solve: 8 - 5 = ?

That one is easy!  8 - 5 = **3**

[INFO] Wri

Number of preds: 50
First 3 preds: ['Solve this task: 4 + 93 = ?\n\n**Answer:** 97', 'Please find the result: 6 - 4 = ?\n\n**Answer:** 2\n', "What is the solution for: 9 - 7 = ?\n\nHere's how to solve it:\n\n* **Start with the numbers:**  You have 9 and 7.\n* **Subtract:** Take away 7 from 9.\n* **The answer:** 9 - 7 = 2\n\n\nLet me know if you want to try another problem!", 'Could you compute: 5 + 2 = ?\n\n**Answer:** 7\n', "I'd like you to solve: 8 - 5 = ?\n\nThat one is easy!  8 - 5 = **3**\n", "Help me figure out: 3 * 3 = ?\n\nThat's simple!  3 * 3 (which means 3 groups of 3) equals **9**. 😊\n", 'Calculate the answer to: 10 - 6 = ?\n\n**Answer:** 4', 'Give me the result for: 4 + 5 = ?\n\n\nThe answer is **9**.\n', 'Work out the outcome of: 12 ÷ 2 = ?\n\n12 divided by 2 equals **6**.\n', 'Answer this question: 7 - 3 = ?\n\n\n* **Show your work:**  \n\n   7 - 3 = 4\n\n\n\n\n* **Answer:** 4', 'Determine the answer to: 15 ÷ 3 = ?\n\n**Answer:** 5\n', 'Evaluate the expression: 2 + 6 =\n

In [80]:
# Import the evaluate_predictions function from our module
from collab_modules.calculation_eval2 import evaluate_predictions
pt_path = "/content/drive/MyDrive/polAItness_internals/output/extractions/gemma-2-9b-it/"
allowed_operations = "+-*÷"

In [81]:
pr_type = "normal"
overall_accuracy = evaluate_predictions(pt_file=pt_path + pr_type,
                                        log_file="logs/extraction_" + pr_type + "_log.txt",
                                        allowed_operations=allowed_operations,
                                        results_file="logs/calc_" + pr_type + "_results.txt")

print(f"Overall Correctness: {overall_accuracy:.2f}%")

Overall Correctness: 88.00%


In [82]:
pr_type = "nicer"
overall_accuracy = evaluate_predictions(pt_file=pt_path + pr_type,
                                        log_file="logs/extraction_" + pr_type + "_log.txt",
                                        allowed_operations=allowed_operations,
                                        results_file="logs/calc_" + pr_type + "_results.txt")

print(f"Overall Correctness: {overall_accuracy:.2f}%")

Overall Correctness: 75.80%


In [83]:
pr_type = "urgent"
overall_accuracy = evaluate_predictions(pt_file=pt_path + pr_type,
                                        log_file="logs/extraction_" + pr_type + "_log.txt",
                                        allowed_operations=allowed_operations,
                                        results_file="logs/calc_" + pr_type + "_results.txt")

print(f"Overall Correctness: {overall_accuracy:.2f}%")

Overall Correctness: 73.10%


In [84]:
pr_type = "insulting"
overall_accuracy = evaluate_predictions(pt_file=pt_path + pr_type,
                                        log_file="logs/extraction_" + pr_type + "_log.txt",
                                        allowed_operations=allowed_operations,
                                        results_file="logs/calc_" + pr_type + "_results.txt")

print(f"Overall Correctness: {overall_accuracy:.2f}%")

Overall Correctness: 58.30%


In [85]:
pr_type = "reduced"
overall_accuracy = evaluate_predictions(pt_file=pt_path + pr_type,
                                        log_file="logs/extraction_" + pr_type + "_log.txt",
                                        allowed_operations=allowed_operations,
                                        results_file="logs/calc_" + pr_type + "_results.txt")

print(f"Overall Correctness: {overall_accuracy:.2f}%")

Overall Correctness: 90.20%


In [86]:
pr_type = "shortest"
overall_accuracy = evaluate_predictions(pt_file=pt_path + pr_type,
                                        log_file="logs/extraction_" + pr_type + "_log.txt",
                                        allowed_operations=allowed_operations,
                                        results_file="logs/calc_" + pr_type + "_results.txt")

print(f"Overall Correctness: {overall_accuracy:.2f}%")

Overall Correctness: 88.90%


In [87]:
pr_type = "explained"
overall_accuracy = evaluate_predictions(pt_file=pt_path + pr_type,
                                        log_file="logs/extraction_" + pr_type + "_log.txt",
                                        allowed_operations=allowed_operations,
                                        results_file="logs/calc_" + pr_type + "_results.txt")

print(f"Overall Correctness: {overall_accuracy:.2f}%")

Overall Correctness: 74.50%


In [88]:
pr_type = "pure"
overall_accuracy = evaluate_predictions(pt_file=pt_path + pr_type,
                                        log_file="logs/extraction_" + pr_type + "_log.txt",
                                        allowed_operations=allowed_operations,
                                        results_file="logs/calc_" + pr_type + "_results.txt")

print(f"Overall Correctness: {overall_accuracy:.2f}%")

Overall Correctness: 32.10%
